In [58]:

import os
import glob
import pandas as pd


def select(data, **kwargs):
    for k, v in kwargs.items():
        data = data[data[k] == v]
    return data


def set_columns(data, **kwargs):
    for k, v in kwargs.items():
        data[k] = v
    return data


def create_df(report_file):
    return set_columns(pd.read_csv(report_file), system=os.path.basename(report_file).removesuffix('.csv'))


def get_group(class_name):
    return class_name.split('.')[-1].removesuffix('ITCase')


def classify(fp, fn, status):
    if status != 'success':
        return "BEHAVIOR"
    else:
        return "PASSED" if (fp + fn == 0) else "PROPAGATION"


reports_dir = '../galette-evaluation/galette-evaluation-tools/target/flow-reports'
pd.set_option('display.max_rows', 100)

In [59]:
report_files = glob.glob(os.path.join(reports_dir, '*.csv'))
df = pd.concat(list(map(create_df, report_files)))
df['group'] = df['class'].apply(get_group)
df['result'] = df[['fp', 'fn', 'status']].apply(lambda x: classify(*x), axis=1)
df['result'] = pd.Categorical(df['result'], categories=['BEHAVIOR', 'PROPAGATION', 'PASSED'])
df.head()

,class,method,name,tp,fp,fn,status,system,group,result
0,edu.neu.ccs.prl.galette.bench.StaticInitialize...,staticInitializer,"staticInitializer(TagManager, FlowChecker)",1,0,0,success,phosphor,StaticInitializer,PASSED
1,edu.neu.ccs.prl.galette.bench.MethodHandleITCase,methodHandlesExplicitCastArguments,methodHandlesExplicitCastArguments(),0,0,2,success,phosphor,MethodHandle,PROPAGATION
2,edu.neu.ccs.prl.galette.bench.MethodHandleITCase,lookupUnreflectSpecial,lookupUnreflectSpecial(),0,0,2,success,phosphor,MethodHandle,PROPAGATION
3,edu.neu.ccs.prl.galette.bench.MethodHandleITCase,lookupUnreflect,lookupUnreflect(),0,0,2,success,phosphor,MethodHandle,PROPAGATION
4,edu.neu.ccs.prl.galette.bench.MethodHandleITCase,methodHandlesArrayElementSetter,methodHandlesArrayElementSetter(),0,0,1,success,phosphor,MethodHandle,PROPAGATION


In [100]:
totals = df.groupby(['group', 'system'])['result'] \
    .count() \
    .rename('total') \
    .reset_index()
table = df.groupby(['group', 'system'])['result'] \
    .value_counts() \
    .reset_index() \
    .merge(totals, on=['group', 'system'], how='left')
table.head()
table = table[table['result'] != 'PASSED']

In [101]:
table = table.pivot(index=['group', 'total'], values=['count'], columns=['system', 'result']) \
    .reorder_levels(axis=1, order=['system', 'result', None]) \
    .sort_index(axis=1) \
    .sort_index(axis=0) \
    .droplevel(2, axis=1) \
    .fillna(0)
table.index.names =[x.title() for x in table.index.names]
table.columns = table.columns.map(lambda l: tuple(map(lambda s: s.title().replace('-', ''), l)))
table.columns.names = [None, None]
styler = table.style.format(precision=0)
display(styler)